In [1]:
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import HoverTool, ColumnDataSource
import itertools
import shapefile
import pandas as pd
import datetime
import requests
import zipfile
try:
    from StringIO import StringIO
except ImportError:
    from io import BytesIO as StringIO
import os

# Get World Country Map Data
# http://www.naturalearthdata.com/features/
shp = None
dbf = None
url = 'http://www.naturalearthdata.com/http//www.naturalearthdata.com/download/110m/cultural/ne_110m_admin_0_countries.zip'
response = requests.get(url)
with zipfile.ZipFile(StringIO(response.content)) as z:
    for fname in z.namelist():
        name, ext = os.path.splitext(fname)
        if ext == '.shp':
            shp = StringIO(z.read(fname)) 
            #shp = z.open(fname)
        elif ext == '.dbf':
            #dbf = z.open(fname)
            dbf = StringIO(z.read(fname))             
        else:
            pass
sf = shapefile.Reader(shp=shp, dbf=dbf)

# Munge map data for bokeh 
lats = []
lons = []
country = []
for shprec in sf.shapeRecords():
    name_long = str(shprec.record[18])
    country.append(name_long)
    lat, lon = map(list, zip(*shprec.shape.points))
    indices = shprec.shape.parts.tolist()
    lat = [lat[i:j] + [float('NaN')] for i, j in zip(indices, indices[1:]+[None])]
    lon = [lon[i:j] + [float('NaN')] for i, j in zip(indices, indices[1:]+[None])]
    lat = list(itertools.chain.from_iterable(lat))
    lon = list(itertools.chain.from_iterable(lon))
    lats.append(lat)
    lons.append(lon)


In [2]:
df = pd.DataFrame({'x': lats, 'y': lons, 'country': country})
cds = ColumnDataSource(df)
p = figure(width=800)
country_patches = p.patches('x', 'y', source=cds, line_color='white')
hover = HoverTool(renderers=[country_patches], tooltips=[('Country', '@country')])
p.add_tools(hover)
show(p)